<a href="https://colab.research.google.com/github/Gr8eye/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

In [0]:
#Always imports first
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel

In [2]:
#Get Data
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2020-05-21 03:36:31--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data.3’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.03s   

2020-05-21 03:36:31 (611 KB/s) - ‘house-votes-84.data.3’ saved [18171/18171]



In [0]:
# Create Dataframe
Cols = ['Party','Handicapped-Infants', 'Water-Projecjt', 'Budget',
        'Physician-Fee-Freeze', 'El-Salvador-Aid', 'Religious Groups',
        'Anti-Satellite-Ban', 'Aid-to-Contras', 'MX-Missile', 'Immigration',
        'Synfuels', 'Education', 'Right-to-Sue', 'Crime', 'Duty-Free', 
        'South-Africa']

In [4]:
HORV84 = pd.read_csv('house-votes-84.data', header=None,
                     names = Cols, na_values="?")
HORV84.head()

,Party,Handicapped-Infants,Water-Projecjt,Budget,Physician-Fee-Freeze,El-Salvador-Aid,Religious Groups,Anti-Satellite-Ban,Aid-to-Contras,MX-Missile,Immigration,Synfuels,Education,Right-to-Sue,Crime,Duty-Free,South-Africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [5]:
#Looking good?
HORV84['Party'].value_counts()

democrat      267
republican    168
Name: Party, dtype: int64

In [0]:
#Clean Data
#Starting with recoding the y/n into numeric values(binary)
HORV84.replace({'y':1, 'n':0}, inplace=True)

In [7]:
#Taking random sample to get a feel for the groups
HORV84.sample(10)

,Party,Handicapped-Infants,Water-Projecjt,Budget,Physician-Fee-Freeze,El-Salvador-Aid,Religious Groups,Anti-Satellite-Ban,Aid-to-Contras,MX-Missile,Immigration,Synfuels,Education,Right-to-Sue,Crime,Duty-Free,South-Africa
233,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0
425,democrat,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN
269,democrat,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
327,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
262,democrat,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,NaN,0.0,1.0,1.0
33,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
182,democrat,1.0,NaN,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
336,democrat,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,NaN,1.0
162,democrat,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0
180,democrat,NaN,NaN,1.0,0.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN


In [0]:
# Setting up to measure Votes...
Reps = HORV84[HORV84['Party']=='republican']
Dems = HORV84[HORV84['Party']=='democrat']

In [0]:
# Function to check alpha 
def check(pvalue):
  "return boolean confirming if alpha < .01"
  return pvalue < .01

## Null Hypothesis: There are few differences between the votes of Democrats and Republicans. 

In [10]:
# *Bill Democrats support more than republicans(p<0.01)
# I expect Democrats to be more supportive of bills such as:
# Budget, Aid-to-Contras, 'South-Africa(X)', 'El-Salvador-Aid(X)'
print(Dems['Aid-to-Contras'].mean())
Reps['Aid-to-Contras'].mean()

0.8288973384030418


0.15286624203821655

In [11]:
ttest_ind(Reps['Aid-to-Contras'],Dems['Aid-to-Contras'], nan_policy='omit')

Ttest_indResult(statistic=-18.052093200819733, pvalue=2.82471841372357e-54)

In [12]:
# Confirming that pvalue is <.01
check(2.82471841372357e-54)

True

In [13]:
# Bill Republicans support more than Democrats(p<0.01)
# I expect Republicans to be more supportive of bills related to:
# Religious Groups, 'Right-to-Sue', 'Crime'
print(Reps['Physician-Fee-Freeze'].mean())
Dems['Physician-Fee-Freeze'].mean()

0.9878787878787879


0.05405405405405406

In [14]:
ttest_ind(Reps['Physician-Fee-Freeze'],Dems['Physician-Fee-Freeze'], nan_policy='omit')

Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)

In [15]:
#Continuing to confirm because I created the function and I'm going to use it!
check(1.994262314074344e-177)

True

In [16]:
# Bill where the Dems and Reps are in equal support (p<0.1)
# I expect their to be equal support in issues: 
# 'Duty-Free'(X), 'Education'(X), 'Anti-Satellite-Ban'(X) Handicapped-Infants(X)
# Synfuels(X), Immigration, Waterbridge
print(Reps['Immigration'].mean())
Dems['Immigration'].mean()

0.5575757575757576


0.4714828897338403

In [17]:
ttest_ind(Reps['Immigration'],Dems['Immigration'], nan_policy='omit')

Ttest_indResult(statistic=1.7359117329695164, pvalue=0.08330248490425066)

In [18]:
check(0.08330248490425066)

False

## Stretch Goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Work on Performing a T-test without using Scipy in order to get "under the hood" and learn more thoroughly about this topic.
### Start with a 1-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://blog.minitab.com/hs-fs/hubfs/Imported_Blog_Media/701f9c0efa98a38fb397f3c3ec459b66.png?width=247&height=172&name=701f9c0efa98a38fb397f3c3ec459b66.png) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then try a 2-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://lh3.googleusercontent.com/proxy/rJJ5ZOL9ZDvKOOeBihXoZDgfk7uv1YsRzSQ1Tc10RX-r2HrRpRLVqlE9CWX23csYQXcTniFwlBg3H-qR8MKJPBGnjwndqlhDX3JxoDE5Yg) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then check your Answers using Scipy!